# Machine Vision and Cognitive Robotics - Exercise 4

<!-- 
Automation and Control Institute - TU Wien
Matthias Hirschmanner 2019
machinevision@acin.tuwien.ac.at
 -->

In this exercise you will implement a neural network to classify images of the CIFAR10 dataset (https://www.cs.toronto.edu/~kriz/cifar.html). It consists of 60000 images with a resolution of 32x32. The different classes and example images for each classes are shown in Figure 1 below. We will use the library Keras with the TensorFlow backend to create our neural networks. More specifically, we will use the Functional API of Keras. 


![Classes of CIFAR10](https://i2.wp.com/appliedmachinelearning.blog/wp-content/uploads/2018/03/cifar2.jpg?resize=427%2C325&ssl=1)

Figure 1: Examples of the different classes of the CIFAR10 dataset.

---



Please keep the code in the sections below clean and only change it in the dedicated areas. You shouldn't need to change it anywhere else. You also shouldn't need to use any additional libraries other than the ones already imported. If you need to change the code somewhere else or need to import a different library to get a functioning program, that might be a bug. Please report it in the Tuwel forum or send us a mail to machinevision@acin.tuwien.ac.at


## Import Libraries
In a first step we import the libraries needed for the exercise. Please execute the cell below.

In [ ]:
# Sets the tensorflow version. If you use conda, you might need to delete this line
import numpy as np
from matplotlib import pyplot as plt
import random
import requests
import imageio
from PIL import Image

from tensorflow.keras.datasets import cifar10
from tensorflow.keras.models import Model
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import callbacks 

import tensorflow.keras.utils 
import tensorflow.keras.backend as K
from tensorflow.python.client import device_lib

# Fixes a crash when running on Mac OS
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

get_ipython().__class__.__name__ = "ZMQInteractiveShell"

print("These devices are available. If no GPU shows up in Google Colab, \
change Runtime type to GPU.")
print(device_lib.list_local_devices())


## Load Dataset
Next we load the CIFAR10 dataset. The training images are in the numpy array `X_train` and the test images in the numpy array `X_test`. The labels are contained in the numpy arrays `y_train`and `y_test`.


In [ ]:
# Load datasetlassify images of the CI
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Print shape of arrays:
print('Training Images Shape:', X_train.shape, 'Data Type:', X_train.dtype)
print('Test Images Shape:', X_test.shape)
print('Training Labels Shape:', y_train.shape)
print('Test Labels Shape:', y_test.shape)

## Plot random images

In [ ]:
#@title Run this cell!
#@markdown It will show you 9 random images of the dataset. 
#@markdown The code itself is not relevant, but it is important for later to execute this cell because some plotting functions are defined here. 
fig, axs = plt.subplots(3, 3,figsize=(10,10))
labels = {0:'airplane',1:'automobile',2:'bird',3:'cat',4:'deer',5:'dog',6:'frog',7:'horse',8:'ship',9:'truck'}
random_indices = np.random.randint(X_test.shape[0], size=9)
for (idx,ax) in zip(random_indices,axs.flat):
  ax.set_title(labels[y_test[idx,0]])
  ax.imshow(X_test[idx])
  
#Function to plot the training
def plot_history(history):

  # Plot the Training History
  fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(20,5))
  fig.suptitle('MLP Training')
  ax1.plot(history.history['accuracy'], label="Train")
  ax1.plot(history.history['val_accuracy'], label="Test")
  ax1.set(xlabel='Epoch', ylabel='Accuracy')
  ax1.legend(loc="upper left")
  ax2.plot(history.history['loss'], label='Train')
  ax2.plot(history.history['val_loss'], label='Test')
  ax2.set(xlabel='Epoch', ylabel='Loss')
  ax2.legend(loc="upper right")


#Function corect/wrong recognition:
def plot_predictions(data, model_name, class_name=None):
  labels = {0:'airplane',1:'automobile',2:'bird',3:'cat',4:'deer',5:'dog',6:'frog',7:'horse',8:'ship',9:'truck'}
  labels_inv = {v: k for k, v in labels.items()}
  y_predictions = model_name.predict(data) 

  # Get the predicted labels [0,9]
  y_predictions_labels = np.argmax(y_predictions,axis=1)
  y_predictions[y_predictions<0.5]=0 # prediction rate R

# getting a list of ALL images and labels with predicion rate R 
  out_ind = np.nonzero(y_predictions) # index of non zero
  out_ind_label = out_ind[1] # index of a label 0-9
  out_ind_image = out_ind[0] # index of an image 0...N (N - size of dataset)
  out_images = X_test[out_ind_image]
# getting lists of ONLY correct and wrong predicted images
  true_out_ind = out_ind[0][y_test[out_ind[0]][:,0] == out_ind_label]
  wrong_out_ind = out_ind[0][y_test[out_ind[0]][:,0] != out_ind_label]
# check if class_name defined at all  (1.'if' if class_name defined, 2.'if' if class_name from the labels list)
  if class_name:
    class_number = labels_inv.get(class_name, -1)
    if class_number != -1: # taking only predictions with defined class_name 

      # Images that were correctly predicted with class_name
      true_out_ind = true_out_ind[y_predictions_labels[true_out_ind]==class_number]
      
      # Images that are part of class_name, but were wrongly predicted
      false_positive_ind = wrong_out_ind[y_predictions_labels[wrong_out_ind]==class_number]

      # Images that are part of class_name but were wrongly predicted
      false_negative_ind = wrong_out_ind[y_test[wrong_out_ind][:,0]==class_number]
      print('Number of correct recognized', class_name, 'images is', len(true_out_ind))
      print('Number of images recognized as', class_name, 'but are not of that class is', len(false_positive_ind))
      print('Number of images not recognized as', class_name, 'but actually are of that class is', len(false_negative_ind))

    else:
      print('No such class like', class_name + '. Check class name again.')
  else:  
    print('Number of correct recognized images is', len(true_out_ind))
    print('Number of wrong recognized images is', len(wrong_out_ind))
# plot of 10 correct and 10 wrong random recognized images 
  true_ind_plot = random.sample(list(true_out_ind), k=10) # correct
  wrong_ind_plot = random.sample(list(wrong_out_ind), k=10) # wrong
  if class_name:
    if class_number != -1:
      fig = plt.figure(figsize=(20,7.5))

      if len(false_positive_ind) > 10:
        false_positive_ind_plot = random.sample(list(false_positive_ind), k=10) # wrong
      else:
        false_positive_ind_plot = false_positive_ind
      
      if len(false_negative_ind) > 10:
        false_negative_ind_plot = random.sample(list(false_negative_ind), k=10) # wrong
      else:
        false_negative_ind_plot = false_negative_ind

      for idx, image_idx in enumerate(true_ind_plot):
        a = fig.add_subplot(3, 10, idx+1)
        imgplot = plt.imshow(X_test[image_idx])
        a.set_title(labels[y_predictions_labels[image_idx]], color='green', y=-0.2)
        a.axis('off')
      for idx, image_idx in enumerate(false_positive_ind_plot):
        a = fig.add_subplot(3, 10, idx+11)
        imgplot = plt.imshow(X_test[image_idx])
        a.set_title(labels[y_predictions_labels[image_idx]], color='red', y=-0.2)
        a.axis('off')
        a.set_xlabel(labels[y_predictions_labels[image_idx]], color='red')
      for idx, image_idx in enumerate(false_negative_ind_plot):
        a = fig.add_subplot(3, 10, idx+21)
        imgplot = plt.imshow(X_test[image_idx])
        a.set_title(labels[y_predictions_labels[image_idx]], color='red', y=-0.2)
        a.axis('off')
      plt.figtext(0.5,0.9, "Correct Predictions", ha="center", va="top", fontsize=14)
      plt.figtext(0.5,0.63, "False Positives", ha="center", va="top", fontsize=14)
      plt.figtext(0.5,0.36, "False Negatives", ha="center", va="top", fontsize=14)

  else:
    fig = plt.figure(figsize=(20,5))
    for idx, image_idx in enumerate(true_ind_plot):
      a = fig.add_subplot(2, 10, idx+1)
      imgplot = plt.imshow(X_test[image_idx])
      a.set_title(labels[y_predictions_labels[image_idx]], color='green')
    for idx, image_idx in enumerate(wrong_ind_plot):
      a = fig.add_subplot(2, 10, idx+11)
      imgplot = plt.imshow(X_test[image_idx])
      a.set_title(labels[y_predictions_labels[image_idx]], color='red')


# Normalization of an image and training
def predict_image(nn_model, image):
  labels = {0:'airplane',1:'automobile',2:'bird',3:'cat',4:'deer',5:'dog',6:'frog',7:'horse',8:'ship',9:'truck'}
  # prediction with a model
  test_prediction = nn_model.predict(image) # define a model of nn
  print(test_prediction)
  # top 5 predictions
  out_ind_up = sorted(range(len(list(test_prediction[0]))), key = lambda sub: test_prediction[0][sub])[-5:] 
  # result
  print("Top-5 Predictions:")
  for i in out_ind_up[::-1]:
    print(' ', labels[i].capitalize(), '-', round((test_prediction[0][i])*100,2), '%') 
  
# Load an image from an external URL an returns it
def load_image_link(link): 
  response = requests.get(link)
  im_from_link = Image.fromarray(imageio.imread((response.content))) 
  # Adjust width and height to CIFAR10 resolution
  width = 32
  height = 32
  loaded_image = im_from_link.resize((width, height), Image.ANTIALIAS)
  loaded_image = np.expand_dims(loaded_image, axis = 0)
  return loaded_image

# Load an image from Colab folder
def load_image_colab(image_name): 
  # Default Colab folder: "/content/..."
  imageFile = "/content/" + image_name # taking a file with original size
  im_from_fold = Image.open(imageFile) # if you do it from colab folder
  # Adjust width and height to CIFAR10 resolution
  width = 32
  height = 32
  loaded_image = np.array(im_from_fold.resize((width, height), Image.ANTIALIAS))
  loaded_image = np.expand_dims(loaded_image, axis = 0)
  return loaded_image


## Normalize Images (1 Point)
Your first task is to create the function `normalize_images` which takes the training images `X_train` and the test images `X_test` as parameters. The outputs should be the two normalized sets with the shape `(dataset_size, 32, 32, 3)` and data type `np.float32`. Normalize in a way, that the returned arrays have approximately zero mean ($\mu=0$) and unit variance ($\sigma=1$). You can decide if you want to normalize per color channel or over the whole dataset. Only use the standard numpy functionality to implement this function (e.g. `np.mean()` and `np.var()`)

**IMPORTANT:** Use the same mean and and standard deviation you get from the training set also for the test set, because we want our training and test data to go through the same transformation.

You can find more detailed explanation of why we do normalization in this video by Andrew Ng: [Andrew Ng - Normalizing Inputs](https://www.youtube.com/watch?v=FDCfw-YqWTE)

In [ ]:
def normalize_images(training_images: np.ndarray, 
                     test_images: np.ndarray) -> Tuple[np.ndarray, np.ndarray]:
  """ Normalize training and test images to have zero mean and unit variance

  :param training_images: The training images
  :type training_images: np.ndarray with shape (train_batch_size, width, height, channels)
    and dtype = np.uint8 and values in the range [0, 255]

  :param test_images: The test images
  :type test_images: np.ndarray with shape (test_batch_size, width, height, channels)
    with dtype = np.uint8 and values in the range [0, 255]
  
  :return: (normalized_training_images, normalized_test_images)
    normalized_training_images: Normalized training images with zero mean and unit variance
  :rtype: Tuple(normalized_training_images, normalized_test_images)
    normalized_training_images: np.ndarray with shape (train_batch_size, width, height, channels)
      and dtype = np.float32
    normalized_test_images: np.ndarray with shape (test_batch_size, width, height, channels)
      and dtype = np.float32
  """
  ########## Fix This Part ##########
  
  normalized_training_images = np.zeros(training_images.shape)
  normalized_test_images = np.zeros(test_images.shape)
  
  ###################################

  return normalized_training_images, normalized_test_images

## Preprocess labels (1 Point)
Keras needs the labels as an one-hot encoded vector for training. The function `preprocess_labels(labels, no_of_classes)` should take a label with shape `(batch_size, 1)`, an integer with the number of classes and create a one-hot encoded numpy array with the shape `(batch_size, number_of_classes)`. Keras has the built-in function `tensorflow.keras.utils.to_categorical` which creates this vector for us. For this exercise, don't use it or similar functions.

In [ ]:
def preprocess_labels(labels: np.ndarray, 
                      no_of_classes: int) -> np.ndarray:
  """ Return one-hot encoded labels

  :param labels: Labels for each training/test image as a number in the range [0, number_of_classes - 1]
  :type labels: np.ndarray with shape (batch_size, 1) and dtype = np.uint8

  :param no_of_classes: Number of classes in the dataset
  :type no_of_classes: int

  :return output_labels: The one-hot encoded labels. Each label is a vector with 
    all entries = 0 and a single entry = 1 at the index of the correct class. 
  :rtype: np.ndarray with shape (batch_size, 10) with dtype = np.float32
  """
  ########## Fix This Part ##########
  
  output_labels = np.zeros((labels.shape[0], no_of_classes))  


  ###################################
  
  return output_labels



The next cell tests your function with the built-in function of Keras:

In [ ]:
y_train_onehot = preprocess_labels(y_train, 10)
y_test_onehot = preprocess_labels(y_test, 10)
if((y_train_onehot == tensorflow.keras.utils.to_categorical(y_train, 10)).all() 
  and (y_test_onehot == tensorflow.keras.utils.to_categorical(y_test,10)).all()):
  print("Test Successful!")
else:
  print("Something went wrong, check your preprocess_labels function")

##Loss Function (1 Point)
We need a loss function to perform the actual training of our model. Implement the cross-entropy loss with the mathematical functions defined in keras.backend. The function `my_crossentropy_loss` should take the input tensor `y_true` with the one-hot-encoded training labels and the predictions of our network `y_pred` which is a tensor of the same dimensions with the softmax prediction values for each class (between 0 and 1). The shape of the input tensors is `(number_of_predictions, number_of_classes)` and the shape of the output tensor should be `(number_of_predictions,)`. You should use the functions provided in the Keras backend e.g. `K.sum()` (https://www.tensorflow.org/api_docs/python/tf/keras/backend/sum). Don't use the built-in crossentropy loss of Keras. For an example check the code snippet below. The cross-entropy loss is defined as 
\begin{equation}
L(\mathbf{y},\mathbf{\hat{y}})=-\sum_{k=1}^{K}y_{k}\log(\hat{y}_{k}).
\end{equation}

This video gives a good explanation about the cross-entropy loss: https://youtu.be/ueO_Ph0Pyqk

In [ ]:
def my_crossentropy_loss(y_true: tf.Tensor, 
                         y_pred: tf.Tensor) -> tf.Tensor:
  """ Calculate the categorical crossentropy loss between the labels and the prediction

  :param y_true: One-hot encoded training labels
  :type y_true: tf.Tensor with shape (batch_size, num_classes) and dtype = np.float32

  :param y_pred: Prediction values of classes in the range [0, 1]
  :type y_pred: tf.Tensor with shape (batch_size, num_classes) and dtype = np.float32

  :return: Categorical crossentropy loss
  :rtype: tf.Tensor with shape (batch_size,) and dtype = np.float32
  """
  ########## Fix This Part ##########

  loss = K.constant([0.,0.,0.],shape=[1,3])
  

  ###################################
  
  return loss

  

You can check your implementation by comparing it to the built-in function of Keras. In the example below we have the prediction and true_labels of 4 images with 3 classes:

In [ ]:

label_tensor = K.constant([1., 0., 0., \
                           0., 1., 0., \
                           0., 0., 1., \
                           0., 0., 1.], shape=[4,3])

prediction_tensor = K.constant([.9,  .05, .05, \
                                .05, .89, .06, \
                                .5,  .4,  .1,  \
                                .01, .01, .98], shape=[4,3])

my_crossentropy_output = my_crossentropy_loss(label_tensor, prediction_tensor)
keras_crossentropy_output = K.categorical_crossentropy(label_tensor, prediction_tensor)

print('Input shape: ', label_tensor.shape, ' Output Shape: ', my_crossentropy_output.shape)
print('My Crossentropy Loss:    ', K.eval(my_crossentropy_output))
print('Keras built-in function: ', K.eval(keras_crossentropy_output))

## Linear Classifier (1 Point)
In a first Step we want to train a simple network with only one layer (no hidden layers). A dense layer takes each input value (each pixel intensity) multiplies it with a weight and adds a bias term for each output neuron. An animation of the process is shown here: https://codelabs.developers.google.com/codelabs/cloud-tensorflow-mnist/#3

To be able to feed the images to the network, we need to reshape them to a vector. The function `vectorize_images(images)` takes as input parameter our images as a `numpy.array` with the shape `(batch_size, height, width, color_channels)` and returns a `numpy.array` with the shape `(batch_size, input_dim)`. You can use the function `np.reshape()`.

In [ ]:
def vectorize_images(images: np.ndarray) -> np.ndarray:
  """ Reshape images to a one-dimensional vectors

  :param images: The input images to be normalized
  :type images: np.ndarray with shape (batch_size, height, width, color_channels)

  :return: The images as one-dimensional arrays
  :rtype: np.ndarray with shape (batch_size, height * width * color_channels)
    and the same datatype as the input images
  """
  ########## Fix This Part ##########
  images_vector = np.array([])

  ###################################

  return images_vector


# First normalize our training and test images
X_train_normalized, X_test_normalized = normalize_images(X_train, X_test)

# Now use vectorize_images to create our vectors
X_train_vector = vectorize_images(X_train_normalized)
X_test_vector = vectorize_images(X_test_normalized)

To create the model please use the functional API of Keras since it is more flexible than the sequential model: https://keras.io/getting-started/functional-api-guide/ 

In this part, you should only need to change the arguments of the functions. You can keep the general structure as it is. 

In [ ]:
########## Fix This Part ##########

#TODO: Please only change the parameters here

# This returns a tensor
inputs = layers.Input() #TODO: Fix the parameters

# A layer instance is callable on a tensor, and returns a tensor
output_layer = layers.Dense()(inputs) #TODO: Fix the parameters

###################################


# This creates a model that includes
# the Input layer and one Dense layer
one_layer_model = Model(inputs=inputs,
                        outputs=output_layer)

# We compile our model with the Stochastic Gradient Descent as optimizer
# and the cross-entropy loss we implemented earlier
one_layer_model.compile(optimizer='sgd',
              loss=my_crossentropy_loss, 
              metrics=['accuracy'])

one_layer_model.summary()

### Actual Training
This command starts the actual training. Make sure you selected the hardware accelarator GPU in Runtime -> Change Runtime type. One epoch should take below 10 seconds.

Run the training for about 30 epochs which should get you to >30% validation accuracy. If the model doesn't go above ~10% (random guessing), check your normalization and the activation function of the `output_layer`.


In [ ]:
history_one_layer=one_layer_model.fit(x=X_train_vector, 
                                      y=y_train_onehot, 
                                      validation_data=(X_test_vector, y_test_onehot), 
                                      epochs=30)  # starts training

We can also plot the the training process. Think a bit about what you are seeing here (you will also need to write about it for the documentation).

In [ ]:
plot_history(history_one_layer)

We can also have a look at some predictions of our model of the test set. The function `plot_predictions(data, model_name, class_name=None)` plots predictions of of random images of the test set with our model. The input data should be X_test preprocessed as the model expects it. By specifying the optional parameter class_name, only correct predictions, false positives and false negatives of this class will be plotted. Feel free to change the parameters below. In the image below you can see an example output of the function with the `class_name='horse'` for a different model.

![alt text](https://owncloud.tuwien.ac.at/index.php/apps/files_sharing/ajax/publicpreview.php?x=1680&y=730&a=true&file=horse_prediction_mlpreg.png&t=0w2rKhrCgTDlMmb&scalingup=0)


In [ ]:
plot_predictions(X_test_vector, one_layer_model)

In [ ]:
plot_predictions(X_test_vector, one_layer_model, class_name='ship')

## Multilayer Perceptron (1 Point)
The last model with only one layer reached appr. 30% of validation accuracy. It is already better than random guessing but not really satisfactory. Now it's time to add the "Deep" to Deep Learning. Create a model with multiple dense layers (a multilayer perceptron) with ReLu activation in the hidden layers and train it again for about 30 Epochs. The exact structure of the model is up to you, a good starting point is a layer with 256 units, followed by another layer with 128 units and followed by the output layer. This model should reach around 50% of validation accuracy. Training time is dependent on your model but should be below 20s per epoch.

In [ ]:
########## Fix This Part ##########

# This returns a tensor
mlp_inputs = layers.Input()

# a layer instance is callable on a tensor, and returns a tensor
mlp_predictions = layers.Dense()()

###################################


mlp_model = Model(inputs = mlp_inputs, outputs = mlp_predictions)
mlp_model.compile(optimizer = 'sgd',
              loss='categorical_crossentropy', # We now use the built-in categorical_crossentropy loss
              metrics=['accuracy'])
history_mlp=mlp_model.fit(x = X_train_vector, 
                          y = y_train_onehot, 
                          validation_data = (X_test_vector, y_test_onehot), 
                          epochs = 30)  # starts training


Time again to plot the training. You should now see the effect of overfitting.

In [ ]:
plot_history(history_mlp)

## MLP with Regularization (1 Point)
To counter the problem of overfitting, you should add measures for regularization. Create a new model based on your MLP from the last point which includes at least one regularization technique. It should improve the validation accuracy slightly.

In [ ]:
########## Fix This Part ##########

mlpreg_inputs = layers.Input()

mlpreg_predictions = layers.Dense()()

###################################

mlpreg_model = Model(inputs = mlpreg_inputs, outputs = mlpreg_predictions)

# We save the weights here before training to reset them in the experiments part later
mlpreg_model.save_weights('mlpreg_model_weights.h5') 

mlpreg_model.compile(optimizer = 'sgd',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
mlpreg_model.summary()
history_mlpreg = mlpreg_model.fit(x = X_train_vector, 
                          y = y_train_onehot, 
                          validation_data = (X_test_vector, y_test_onehot), 
                          epochs = 30)  # starts training



Let's plot the training process again. If the overfitting is not solved, add additional regularization techniques.

In [ ]:
plot_history(history_mlpreg)

We can again plot some predictions.

In [ ]:
plot_predictions(X_test_vector, mlpreg_model, class_name=None)

## Convolutional Neural Network (2 Points)
When converting our images to vectors, all spatial information is lost. A convolutional neural network takes matrices (tensors to be more precise) as input and computes features which make use of spatial information. In this exercise implement a neural network as specified in the image below. Add regularization techniques until you reach at least 75% of validation accuracy with a training time of below 30s per epoch.

Notes: 3x3 Conv X means a Conv2D layer with kernel 3x3 and X units. 
Maxpool /2 means a MaxPool layer with kernel=(2,2) and stride=2.


![Network Architecture](https://owncloud.tuwien.ac.at/index.php/apps/files_sharing/ajax/publicpreview.php?x=3360&y=1178&a=true&file=CNN-architecture.png&t=Ms797G0SEXgeZZe&scalingup=0)

In [ ]:
X_train_normalized, X_test_normalized = normalize_images(X_train,X_test)

########## Fix This Part ##########

cnn_inputs = layers.Input()

cnn_predictions = layers.Dense()()

###################################

cnn_model = Model(inputs=cnn_inputs, outputs=cnn_predictions)

# We use the Adam optimizer here which uses more complex methods than our standard SGD
cnn_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

cnn_model.summary()
history_cnn=cnn_model.fit(x=X_train_normalized, 
                          y=y_train_onehot, 
                          validation_data=(X_test_normalized, y_test_onehot), 
                          epochs=30)  # starts training

Plot the training again.

In [ ]:
plot_history(history_cnn)

We can again plot some predictions.

In [ ]:
plot_predictions(X_test_normalized, cnn_model, class_name=None)

##Finetuning (2 Points)
In the next step we want to use a model which is already pretrained on a large dataset (Imagenet) and finetune it for the CIFAR10 dataset. Keras already provides us with multiple pretrained models (see https://keras.io/applications/). Your task is to take the ResNet50 model and **only** retrain the last layer. You should upsample the images first because ResNet is trained on Images with the resolution of (224, 224). However training with this resolution takes very long with not that much gain. A resolution of (128, 128) is a good compromise between accuracy and training time. You can use the `UpSampling2D` layer of Keras. You should also preprocess the images the same way as ResNet using the function `preprocess_input` from the ResNet50 library. This approach should get you to over 80% of validation accuracy. Training time will take much longer here (~120 seconds per epoch), so we will only train for 5 epochs. 

In [ ]:
from tensorflow.keras.applications.resnet import ResNet50, preprocess_input


########## Fix This Part ##########

X_train_resnet = np.zeros((10,32,32,3))
X_test_resnet = np.zeros((10,32,32,3))

resnet_input = layers.Input()
resnet_predictions = layers.Dense()()

###################################

resnet_model = Model(inputs=resnet_input, outputs=resnet_predictions)
resnet_model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

resnet_model.summary()
history_resnet=resnet_model.fit(x=X_train_resnet, 
                          y=y_train_onehot, 
                          validation_data=(X_test_resnet, y_test_onehot),
                          epochs=5)  # starts training

In [ ]:
plot_history(history_resnet)

# Experiments
Here you can add code that you need for your documentation. Feel free to play around in this section with the models, your own images, create new models, data augmentation, different datasets, ...



In [ ]:
# Create an optimizer and then reload weights we saved earlier.
my_sgd = optimizers.SGD()
mlpreg_model.load_weights('my_model_weights.h5')

In [ ]:
# Load an image from that link and get Top-5 predictions
test_im = load_image_link('https://www.telegraph.co.uk/content/dam/news/2019/01/12/TELEMMGLPICT000185347942_trans_NvBQzQNjv4Bq4k9pB6mVv575RZMUuuHUNjlaTMTxUhlzF8Rkw038U-A.jpeg?imwidth=1400')
plt.imshow(test_im[0,...])
test_im = normalize_images(X_train, test_im)[1]
vec_im = vectorize_images(test_im)
predict_image(one_layer_model, vec_im)


In [ ]:
# If you want to mount your Google Drive to store weights or plots for example
# you can use the commands below. Check this link for more information:
# https://colab.research.google.com/notebooks/io.ipynb#scrollTo=u22w3BFiOveA
from google.colab import drive
drive.mount('/content/drive')